In [3]:
!pip install requests beautifulsoup4 transformers selenium
!apt-get update
!apt install chromium-chromedriver


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.7/475.7 kB 21.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.0 MB/s eta 0:00:00
Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:5 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists..

In [8]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from transformers import pipeline
import sqlite3
import os


# Setup ChromeDriver options for Colab
chrome_options = Options()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

# Path to ChromeDriver executable compatible with Colab
chrome_driver_path = '/usr/lib/chromium-browser/chromedriver'

# Initialize ChromeDriver correctly
driver = webdriver.Chrome(options=chrome_options)

# Function to fetch web content using requests and BeautifulSoup
def fetch_web_content(url):
    response = requests.get(url)
    response.raise_for_status()
    return response.text

# Function to scrape content using BeautifulSoup
def scrape_content(html_content):
    soup = BeautifulSoup(html_content, 'html.parser')
    paragraphs = soup.find_all('p')
    content = ' '.join([para.get_text() for para in paragraphs])
    return content

# Function to take screenshot using Selenium
def take_screenshot(url, filename):
    driver.get(url)
    driver.save_screenshot(filename)

# Function to generate content summary using Hugging Face's pipeline
def generate_summary(content):
    summarizer = pipeline("summarization")
    summary = summarizer(content, max_length=150, min_length=30, do_sample=False)[0]['summary_text']
    return summary

# Function to initialize SQLite database
def initialize_database():
    conn = sqlite3.connect('workflow_results.db')
    cursor = conn.cursor()
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS workflow_results (
            url TEXT PRIMARY KEY,
            scraped_content TEXT,
            llm_summary TEXT,
            screenshot_path TEXT
        )
    ''')
    conn.commit()
    conn.close()

# Function to store results in SQLite database
def store_results(url, scraped_content, llm_summary, screenshot_path):
    conn = sqlite3.connect('workflow_results.db')
    cursor = conn.cursor()
    cursor.execute('''
        INSERT OR REPLACE INTO workflow_results (url, scraped_content, llm_summary, screenshot_path)
        VALUES (?, ?, ?, ?)
    ''', (url, scraped_content, llm_summary, screenshot_path))
    conn.commit()
    conn.close()

# Function to fetch cached results from SQLite database
def fetch_cached_results(url):
    conn = sqlite3.connect('workflow_results.db')
    cursor = conn.cursor()
    cursor.execute('''
        SELECT scraped_content, llm_summary, screenshot_path FROM workflow_results WHERE url=?
    ''', (url,))
    row = cursor.fetchone()
    conn.close()
    return row

def handle_workflow(url):
    # Check if results are cached
    cached_results = fetch_cached_results(url)
    if cached_results:
        scraped_content, llm_summary, screenshot_path = cached_results
        print("Results fetched from cache:")
    else:
        # Fetch web content and scrape using BeautifulSoup
        html_content = fetch_web_content(url)
        scraped_content = scrape_content(html_content)[:1024]  # Truncate to 1024 tokens

        # Take screenshot
        screenshot_filename = 'screenshot.png'
        take_screenshot(url, screenshot_filename)
        screenshot_path = os.path.abspath(screenshot_filename)

        # Generate LLM summary
        llm_summary = generate_summary(scraped_content)

        # Store results in database
        store_results(url, scraped_content, llm_summary, screenshot_path)
        print("Results generated and stored in database:")

    # Print results or further process as required
    print(f"Scraped Content:\n{scraped_content}\n")
    print(f"LLM Generated Summary:\n{llm_summary}\n")
    print(f"Screenshot Path: {screenshot_path}")

# Example usage
if __name__ == "__main__":
    url = 'https://en.wikipedia.org/wiki/Generative_artificial_intelligence'
    handle_workflow(url)



No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.


Results generated and stored in database:
Scraped Content:

 Generative artificial intelligence (generative AI, GenAI,[1] or GAI) is artificial intelligence capable of generating text, images, videos, or other data using generative models,[2] often in response to prompts.[3][4] Generative AI models learn the patterns and structure of their input training data and then generate new data that has similar characteristics.[5][6]
 Improvements in transformer-based deep neural networks, particularly large language models (LLMs), enabled an AI boom of generative AI systems in the early 2020s. These include chatbots such as ChatGPT, Copilot, Gemini and LLaMA, text-to-image artificial intelligence image generation systems such as Stable Diffusion, Midjourney and DALL-E, and text-to-video AI generators such as Sora.[7][8][9][10] Companies such as OpenAI, Anthropic, Microsoft, Google, and Baidu as well as numerous smaller firms have developed generative AI models.[3][11][12]
 Generative AI has us